# the graphviz part

In [5]:
from graphviz import Digraph
import json

In [6]:
def gen_graph(explore_name, join):
    dot = Digraph(f'{explore_name}')
    dot.node(explore_name, explore_name)
    for join in join:
        dot.node(join, join)
        dot.edge(explore_name, join)
    dot.render(f'{explore_name}.gv', view=True)
    

In [7]:
# for lineage_map in os.listdir('maps'):
with open(f'maps/map_salesforce_sf__cases.json') as f:
    lineage_jsons = json.load(f)

    gen_graph(explore_name=lineage_jsons['explore_name'], join=lineage_jsons['explore_joins'])
#         break
        

In [8]:
lineage_jsons

{'explore_name': 'sf__cases',
 'explore_joins': ['sf__cases',
  'sf__case_comments',
  'sf__accounts',
  'sf__opportunities',
  'sf__opportunity_products',
  'sf__opportunity_revenue_movements',
  'sf__opportunity_total_revenue',
  'sf__contacts',
  'sf__users',
  'sf__users',
  'sf__leads']}

# final

## main

In [33]:
dir_path = '..'

In [39]:
from graphviz import Digraph
import json
import os
import get_connections
import logging

def gen_graph(explore, explore_name, join_list, view_source_payload, dir_path):
    dot = Digraph(f'{explore_name}')
    dot.node(explore_name, explore_name)

    for join in join_list:

        join_node = f'view_{join}'
        dot.node(join_node, join_node)
        dot.edge(explore_name, join_node)

        view_list = explore[join]

        for view in view_list:

            view_node = f"base_view_{view['view_name']}"
            dot.node(view_node, view_node)
            dot.edge(join_node, view_node)

            source_node = f"{view_source_payload[view['view_name']]}"
            dot.node(source_node, source_node)
            dot.edge(view_node, source_node)

    dot.render(f'{dir_path}/../../graphs/testing.gv', view=True)
    return logging.info(f'Successfully generated dependency graph for Explore {explore_name}.')


def explore_view_dependency(join_list, dir_path):
    
    explore = dict()
    for join in join_list:
        explore_view_list = []
        for file in os.listdir(f'{dir_path}/../maps'):
                if file.startswith('view') and file.split('-')[2] == join:
                    with open(f'{dir_path}/../maps/{file}') as f:
                        view_dict = json.load(f)

                    explore_view_list.append(view_dict)

        explore[join] = explore_view_list

    return explore


def view_source_dependency(source_file, dir_path):
    with open(f'{dir_path}/../maps/{source_file}', 'r') as f:
        view_source = json.load(f)
    
    return view_source


In [41]:
with open('../../maps/explore-salesforce-sf__accounts.json', 'r') as f:
        map_accounts_explore = json.load(f)

explore = explore_view_dependency(join_list = map_accounts_explore['explore_joins'], dir_path=dir_path)

view_source = view_source_dependency('sf__accounts-source.json', dir_path=dir_path)


In [51]:
map_accounts_explore['explore_joins']

['sf__accounts', 'sf__users', 'sf__contacts', 'sf__cases', 'sf__users']

In [52]:
explore = dict()
for join in join_list:
    explore_view_list = []
    for file in os.listdir(f'{dir_path}/../maps'):
            if file.startswith('view') and file.split('-')[1] == join:
                with open(f'{dir_path}/../maps/{file}') as f:
                    view_dict = json.load(f)

                explore_view_list.append(view_dict)

    explore[join] = explore_view_list

In [53]:
gen_graph(explore=explore, explore_name='sf__accounts', join_list=map_accounts_explore['explore_joins'], \
          view_source_payload=view_source, dir_path=dir_path)


In [43]:
map_accounts_explore

{'explore_name': 'sf__accounts',
 'explore_joins': ['sf__accounts',
  'sf__users',
  'sf__contacts',
  'sf__cases',
  'sf__users'],
 'conn': 'data_warehouse'}

In [44]:
explore

{'sf__accounts': [], 'sf__users': [], 'sf__contacts': [], 'sf__cases': []}

In [45]:
view_source

{'sfbase__accounts': 'Amazon.production.salesforce.accounts',
 'sf__accounts': 'Amazon.production.salesforce.accounts',
 'snowflake_sf__accounts': 'SEGMENT.SALESFORCE.ACCOUNTS',
 'total_active_node_count': 'derived table',
 'sf__cases': 'Amazon.production.salesforce.cases',
 'sfbase__users': 'Amazon.production.salesforce.users',
 'sf__contacts': 'Amazon.production.salesforce.contacts',
 'sfbase__cases': 'Amazon.production.salesforce.cases',
 'sf__users': 'Amazon.production.salesforce.users',
 'sfbase__contacts': 'Amazon.production.salesforce.contacts'}

In [46]:
explore=explore
explore_name='sf__accounts'
join_list=map_accounts_explore['explore_joins']
view_source_payload=view_source


In [47]:
dot = Digraph(f'{explore_name}')
dot.node(explore_name, explore_name)


In [48]:
view_source_payload

{'sfbase__accounts': 'Amazon.production.salesforce.accounts',
 'sf__accounts': 'Amazon.production.salesforce.accounts',
 'snowflake_sf__accounts': 'SEGMENT.SALESFORCE.ACCOUNTS',
 'total_active_node_count': 'derived table',
 'sf__cases': 'Amazon.production.salesforce.cases',
 'sfbase__users': 'Amazon.production.salesforce.users',
 'sf__contacts': 'Amazon.production.salesforce.contacts',
 'sfbase__cases': 'Amazon.production.salesforce.cases',
 'sf__users': 'Amazon.production.salesforce.users',
 'sfbase__contacts': 'Amazon.production.salesforce.contacts'}

In [49]:
for join in join_list:

    join_node = f'view_{join}'
    dot.node(join_node, join_node)
    dot.edge(explore_name, join_node)

    view_list = explore[join]

    for view in view_list:

        view_node = f"base_view_{view['view_name']}"
        dot.node(view_node, view_node)
        dot.edge(join_node, view_node)

        source_node = f"{view_source_payload[view['view_name']]}"
        dot.node(source_node, source_node)
        dot.edge(view_node, source_node)


In [50]:
dot.render(f'../../graphs/testing.gv', view=True)

'../../graphs/testing.gv.pdf'